In [1]:
# To install dependendcies
!pip install -r requirements.txt

     |████████████████████████████████| 8.8 MB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 156 kB 18.1 MB/s eta 0:00:01
  Using cached numpy-1.22.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
     |████████████████████████████████| 717 kB 13.9 MB/s eta 0:00:01
  Using cached olefile-0.46.zip (112 kB)
     |████████████████████████████████| 158 kB 14.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 19.5 MB/s eta 0:00:01
     |████████████████████████████████| 48 kB 2.4 MB/s eta 0:00:011
     |████████████████████████████████| 881.9 MB 1.8 kB/s eta 0:00:017     |████████████████▏               | 445.4 MB 2.4 MB/s eta 0:03:05     |███████████████████████▋        | 652.0 MB 9.1 MB/s eta 0:00:26
  Using cached typing_extensions-4.1.1-py3-none-any.whl (26 kB)
     |████████████████████████████████| 99 kB 2.1 MB/s  eta 0:00:01
Using legacy setup.py install for docopt, since package 'wheel' is not installed.
Using legacy setup.py install

In [21]:
#to extract data
!tar -xjvf ./data/data.tar.bz2 -C ./data

data/
data/test_tok.jsonl
data/test_tok.tables.jsonl
data/dev.db
data/train_tok.jsonl
data/test.db
data/train_tok.tables.jsonl
data/dev_tok.jsonl
data/train.db
data/dev_tok.tables.jsonl
data_resplit/
data_resplit/table.db
data_resplit/test.jsonl
data_resplit/tables.jsonl
data_resplit/train.jsonl
data_resplit/dev.jsonl


In [ ]:
!wget -P ./glove http://nlp.stanford.edu/data/wordvecs/glove.42B.300d.zip

--2022-03-22 09:50:40--  http://nlp.stanford.edu/data/wordvecs/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/wordvecs/glove.42B.300d.zip [following]
--2022-03-22 09:50:41--  https://nlp.stanford.edu/data/wordvecs/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.42B.300d.zip [following]
--2022-03-22 09:50:43--  http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Le

In [2]:
!unzip ./glove/glove.42B.300d.zip -d ./glove

Archive:  ./glove/glove.42B.300d.zip
  inflating: ./glove/glove.42B.300d.txt  


In [4]:
!python extract_vocab.py

/home/apoorv/.local/lib/python3.8/site-packages/pandas/compat/__init__.py:109: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
Loading from original dataset
Loading data from data/data/train_tok.jsonl
Loading data from data/data/train_tok.tables.jsonl
Loading data from data/data/dev_tok.jsonl
Loading data from data/data/dev_tok.tables.jsonl
Loading data from data/data/test_tok.jsonl
Loading data from data/data/test_tok.tables.jsonl
Loading word embedding from glove/glove.42B.300d.txt
Killed


In [6]:
!pip install records

You should consider upgrading via the '/home/apoorv/.pyenv/versions/3.8.5/bin/python3.8 -m pip install --upgrade pip' command.


In [13]:
import json
import torch
from sqlnet.utils import *
from sqlnet.model.seq2sql import Seq2SQL
from sqlnet.model.sqlnet import SQLNet
import numpy as np
import datetime

In [14]:
N_word=300
B_word=42
# False in case in which you want to use full dataset
USE_SMALL=True
GPU=True
BATCH_SIZE=16
TRAIN_ENTRY=(True, True, True)  # (AGG, SEL, COND)
TRAIN_AGG, TRAIN_SEL, TRAIN_COND = TRAIN_ENTRY
learning_rate = 1e-3

In [15]:
def load_data(sql_paths, table_paths, use_small=False):
    if not isinstance(sql_paths, list):
        sql_paths = (sql_paths, )
    if not isinstance(table_paths, list):
        table_paths = (table_paths, )
    sql_data = []
    table_data = {}

    max_col_num = 0
    for SQL_PATH in sql_paths:
        print("Loading data from %s"%SQL_PATH)
        with open(SQL_PATH) as inf:
            for idx, line in enumerate(inf):
                if use_small and idx >= 1000:
                    break
                sql = json.loads(line.strip())
                sql_data.append(sql)

    for TABLE_PATH in table_paths:
        print("Loading data from %s"%TABLE_PATH)
        with open(TABLE_PATH) as inf:
            for line in inf:
                tab = json.loads(line.strip())
                table_data[tab[u'id']] = tab

    for sql in sql_data:
        assert sql[u'table_id'] in table_data

    return sql_data, table_data
def load_dataset(dataset_id, use_small=False):
    if dataset_id == 0:
        print("Loading from original dataset")
        sql_data, table_data = load_data('data/data/train_tok.jsonl',
                'data/data/train_tok.tables.jsonl', use_small=use_small)
        val_sql_data, val_table_data = load_data('data/data/dev_tok.jsonl',
                'data/data/dev_tok.tables.jsonl', use_small=use_small)
        test_sql_data, test_table_data = load_data('data/data/test_tok.jsonl',
                'data/data/test_tok.tables.jsonl', use_small=use_small)
        TRAIN_DB = 'data/data/train.db'
        DEV_DB = 'data/data/dev.db'
        TEST_DB = 'data/data/test.db'
    else:
        print("Loading from re-split dataset")
        sql_data, table_data = load_data('data/data_resplit/train.jsonl',
                'data/data_resplit/tables.jsonl', use_small=use_small)
        val_sql_data, val_table_data = load_data('data/data_resplit/dev.jsonl',
                'data/data_resplit/tables.jsonl', use_small=use_small)
        test_sql_data, test_table_data = load_data('data/data_resplit/test.jsonl',
                'data/data_resplit/tables.jsonl', use_small=use_small)
        TRAIN_DB = 'data/data_resplit/table.db'
        DEV_DB = 'data/data_resplit/table.db'
        TEST_DB = 'data/data_resplit/table.db'

    return sql_data, table_data, val_sql_data, val_table_data,\
            test_sql_data, test_table_data, TRAIN_DB, DEV_DB, TEST_DB

In [16]:
def load_word_emb(file_name, load_used=False, use_small=False):
    if not load_used:
        print ('Loading word embedding from %s'%file_name)
        ret = {}
        with open(file_name) as inf:
            for idx, line in enumerate(inf):
                if (use_small and idx >= 5000):
                    break
                info = line.strip().split(' ')
                if info[0].lower() not in ret:
                    ret[info[0]] = np.array(map(lambda x:float(x), info[1:]))
        return ret
    else:
        print ('Load used word embedding')
        with open('glove/word2idx.json') as inf:
            w2i = json.load(inf)
        with open('glove/usedwordemb.npy') as inf:
            word_emb_val = np.load(inf)
        return w2i, word_emb_val

In [17]:
sql_data, table_data, val_sql_data, val_table_data, \
        test_sql_data, test_table_data, \
        TRAIN_DB, DEV_DB, TEST_DB = load_dataset(
                0, use_small=USE_SMALL)

word_emb = load_word_emb('./glove/glove.%dB.%dd.txt'%(B_word,N_word), \
        True, use_small=USE_SMALL)

Loading from original dataset
Loading data from data/data/train_tok.jsonl
Loading data from data/data/train_tok.tables.jsonl
Loading data from data/data/dev_tok.jsonl
Loading data from data/data/dev_tok.tables.jsonl
Loading data from data/data/test_tok.jsonl
Loading data from data/data/test_tok.tables.jsonl
Load used word embedding


FileNotFoundError: [Errno 2] No such file or directory: 'glove/word2idx.json'

In [ ]:
def best_model_name(for_load=False):
    new_data = 'old'
    mode = 'sqlnet'
    train_emb = False
    if for_load:
        use_emb = use_rl = ''
    else:
        use_emb = ''
        use_rl = ''
    use_ca = '_ca'

    agg_model_name = 'saved_model/%s_%s%s%s.agg_model'%(new_data,
            mode, use_emb, use_ca)
    sel_model_name = 'saved_model/%s_%s%s%s.sel_model'%(new_data,
            mode, use_emb, use_ca)
    cond_model_name = 'saved_model/%s_%s%s%s.cond_%smodel'%(new_data,
            mode, use_emb, use_ca, use_rl)

    if not for_load and train_emb:
        agg_embed_name = 'saved_model/%s_%s%s%s.agg_embed'%(new_data,
                mode, use_emb, use_ca)
        sel_embed_name = 'saved_model/%s_%s%s%s.sel_embed'%(new_data,
                mode, use_emb, use_ca)
        cond_embed_name = 'saved_model/%s_%s%s%s.cond_embed'%(new_data,
                mode, use_emb, use_ca)

        return agg_model_name, sel_model_name, cond_model_name,\
                agg_embed_name, sel_embed_name, cond_embed_name
    else:
        return agg_model_name, sel_model_name, cond_model_name

In [ ]:
model = SQLNet(word_emb, N_word=N_word, use_ca=True,
                gpu=GPU, trainable_emb = False)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(),
            lr=learning_rate, weight_decay = 0)

In [ ]:
agg_m, sel_m, cond_m = best_model_name()
train_emb = False
suffix = "trained"

In [19]:
init_acc = epoch_acc(model, BATCH_SIZE,
                val_sql_data, val_table_data, TRAIN_ENTRY)
best_agg_acc = init_acc[1][0]
best_agg_idx = 0
best_sel_acc = init_acc[1][1]
best_sel_idx = 0
best_cond_acc = init_acc[1][2]
best_cond_idx = 0
print('Init dev acc_qm: %s\n  breakdown on (agg, sel, where): %s'%\
        init_acc)
if TRAIN_AGG:
    torch.save(model.agg_pred.state_dict(), agg_m)
    if train_emb:
        torch.save(model.agg_embed_layer.state_dict(), agg_e)
if TRAIN_SEL:
    torch.save(model.sel_pred.state_dict(), sel_m)
    if train_emb:
        torch.save(model.sel_embed_layer.state_dict(), sel_e)
if TRAIN_COND:
    torch.save(model.cond_pred.state_dict(), cond_m)
    if train_emb:
        torch.save(model.cond_embed_layer.state_dict(), cond_e)
for i in range(100):
    print('Epoch %d @ %s'%(i+1, datetime.datetime.now()))
    print(' Loss = %s'%epoch_train(
            model, optimizer, BATCH_SIZE, 
            sql_data, table_data, TRAIN_ENTRY))
    print(' Train acc_qm: %s\n   breakdown result: %s'%epoch_acc(
            model, BATCH_SIZE, sql_data, table_data, TRAIN_ENTRY))
    #val_acc = epoch_token_acc(model, BATCH_SIZE, val_sql_data, val_table_data, TRAIN_ENTRY)
    val_acc = epoch_acc(model,
            BATCH_SIZE, val_sql_data, val_table_data, TRAIN_ENTRY)
    print (' Dev acc_qm: %s\n   breakdown result: %s'%val_acc)
    if TRAIN_AGG:
        if val_acc[1][0] > best_agg_acc:
            best_agg_acc = val_acc[1][0]
            best_agg_idx = i+1
            torch.save(model.agg_pred.state_dict(),
                'saved_model/epoch%d.agg_model%s'%(i+1, suffix))
            torch.save(model.agg_pred.state_dict(), agg_m)
            if train_emb:
                torch.save(model.agg_embed_layer.state_dict(),
                'saved_model/epoch%d.agg_embed%s'%(i+1,suffix))
                torch.save(model.agg_embed_layer.state_dict(), agg_e)
    if TRAIN_SEL:
        if val_acc[1][1] > best_sel_acc:
            best_sel_acc = val_acc[1][1]
            best_sel_idx = i+1
            torch.save(model.sel_pred.state_dict(),
                'saved_model/epoch%d.sel_model%s'%(i+1, suffix))
            torch.save(model.sel_pred.state_dict(), sel_m)
            if train_emb:
                torch.save(model.sel_embed_layer.state_dict(),
                'saved_model/epoch%d.sel_embed%s'%(i+1, suffix))
                torch.save(model.sel_embed_layer.state_dict(), sel_e)
    if TRAIN_COND:
        if val_acc[1][2] > best_cond_acc:
            best_cond_acc = val_acc[1][2]
            best_cond_idx = i+1
            torch.save(model.cond_pred.state_dict(),
                'saved_model/epoch%d.cond_model%s'%(i+1,suffix))
            torch.save(model.cond_pred.state_dict(), cond_m)
            if train_emb:
                torch.save(model.cond_embed_layer.state_dict(),
                'saved_model/epoch%d.cond_embed%s'%(i+1, suffix))
                torch.save(model.cond_embed_layer.state_dict(), cond_e)
    print(' Best val acc = %s, on epoch %s individually'%(
            (best_agg_acc, best_sel_acc, best_cond_acc),
            (best_agg_idx, best_sel_idx, best_cond_idx)))


NameError: name 'model' is not defined

In [20]:
#for testing
TEST_ENTRY=(True, True, True)  # (AGG, SEL, COND)

sql_data, table_data, val_sql_data, val_table_data, \
        test_sql_data, test_table_data, \
        TRAIN_DB, DEV_DB, TEST_DB = load_dataset(
                0, use_small=USE_SMALL)

word_emb = load_word_emb('glove/glove.%dB.%dd.txt'%(B_word,N_word), \
    load_used=True, use_small=USE_SMALL) # load_used can speed up loading

Loading from original dataset
Loading data from data/data/train_tok.jsonl
Loading data from data/data/train_tok.tables.jsonl
Loading data from data/data/dev_tok.jsonl
Loading data from data/data/dev_tok.tables.jsonl
Loading data from data/data/test_tok.jsonl
Loading data from data/data/test_tok.tables.jsonl
Load used word embedding


FileNotFoundError: [Errno 2] No such file or directory: 'glove/word2idx.json'

In [ ]:
model = SQLNet(word_emb, N_word=N_word, use_ca=True, gpu=GPU,
                trainable_emb = False)

In [ ]:
agg_m, sel_m, cond_m = best_model_name()
print("Loading from %s"%agg_m)
model.agg_pred.load_state_dict(torch.load(agg_m))
print("Loading from %s"%sel_m)
model.sel_pred.load_state_dict(torch.load(sel_m))
print("Loading from %s"%cond_m)
model.cond_pred.load_state_dict(torch.load(cond_m))

In [ ]:
print("Dev acc_qm: %s;\n  breakdown on (agg, sel, where): %s"%epoch_acc(
            model, BATCH_SIZE, val_sql_data, val_table_data, TEST_ENTRY))
print("Dev execution acc: %s"%epoch_exec_acc(
        model, BATCH_SIZE, val_sql_data, val_table_data, DEV_DB))
print("Test acc_qm: %s;\n  breakdown on (agg, sel, where): %s"%epoch_acc(
        model, BATCH_SIZE, test_sql_data, test_table_data, TEST_ENTRY))
print("Test execution acc: %s"%epoch_exec_acc(
        model, BATCH_SIZE, test_sql_data, test_table_data, TEST_DB))